In [0]:
from pyspark.sql import SparkSession

new_bronze_path = '/Volumes/workspace/default/my_volume/Sentiment_Project/input_files/new_dataset/'
new_bronze_table_path = '/Volumes/workspace/default/my_volume/Sentiment_Project/enhancements/bronze_tweets'
new_checkpoints_path = '/Volumes/workspace/default/my_volume/Sentiment_Project/enhancements/checkpoints/bronze'

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, when, to_timestamp, hour, date_format

demographic_schema = StructType([
    StructField('text_id', StringType(), True),
    StructField('text', StringType(), True),
    StructField('Selected_text', StringType(), True),
    StructField('sentiment', StringType(), True),
    StructField('Time_of_Tweet', StringType(), True),
    StructField('Age_of_User', StringType(), True),
    StructField('Country', StringType(), True),
    StructField('Population_2020', IntegerType(), True),
    StructField('Land_Area_km2', IntegerType(), True),
    StructField('Density_P_km2', IntegerType(), True)
])

In [0]:
demographic_df = (
    spark.readStream.format('cloudFiles')
    .option('cloudFiles.format', 'csv')
    .option('header', 'true')
    .schema(demographic_schema)
    .load(new_bronze_path)
)

In [0]:
(
    demographic_df.writeStream
    .format('delta')
    .outputMode('append')
    .option('checkpointLocation', new_checkpoints_path)
    .trigger(availableNow = True)
    .start(new_bronze_table_path)
)

In [0]:
bronze_df = spark.read.format('delta').load(new_bronze_table_path)

In [0]:
print(f"Total records: {bronze_df.count()}")
display(bronze_df.limit(10))